Copyright (c) Microsoft Corporation.

Licensed under the MIT License.

# Text2SQL with Semantic Kernel & Azure OpenAI

This notebook demonstrates how the SQL plugin can be integrated with Semantic Kernel and Azure OpenAI to answer questions from the database based on the schemas provided. 

A multi-shot approach is used for SQL generation for more reliable results and reduced token usage. More details can be found in the README.md.

In [1]:
import logging
import os
import yaml
import dotenv
import json
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
)
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.kernel import Kernel
from plugins.prompt_based_sql_plugin.prompt_based_sql_plugin import PromptBasedSQLPlugin
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.prompt_template.prompt_template_config import PromptTemplateConfig
from IPython.display import display, Markdown

logging.basicConfig(level=logging.INFO)

## Kernel Setup

In [2]:
dotenv.load_dotenv()
kernel = Kernel()

## Set up GPT connections

In [3]:
service_id = "chat"

In [4]:
chat_service = AzureChatCompletion(
    service_id=service_id,
    deployment_name=os.environ["OpenAI__CompletionDeployment"],
    endpoint=os.environ["OpenAI__Endpoint"],
    api_key=os.environ["OpenAI__ApiKey"],
)
kernel.add_service(chat_service)

In [5]:
# Register the SQL Plugin with the Database name to use.
sql_plugin = PromptBasedSQLPlugin(database=os.environ["Text2Sql__DatabaseName"])
kernel.add_plugin(sql_plugin, "SQL")

KernelPlugin(name='SQL', description=None, functions={'GetEntitySchema': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='GetEntitySchema', plugin_name='SQL', description='Get the detailed schema of an entity in the Database. Use the entity and the column returned to formulate a SQL query. The view name or table name must be one of the ENTITY NAMES defined in the [ENTITIES LIST]. Only use the column names obtained from GetEntitySchema() when constructing a SQL query, do not make up column names.', parameters=[KernelParameterMetadata(name='entity_name', description='The view or table name to get the schema for. It must be one of the ENTITY NAMES defined in the [ENTITIES LIST] function.', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'The view or table name to get the schema for. It must be one of the ENTITY NAMES defined in the [ENTITIES LIST] function.'}, function_schema_include=True)], is_promp

## Prompt Setup

In [6]:
# Load prompt and execution settings from the file
with open("./prompt.yaml", "r") as file:
    data = yaml.safe_load(file.read())
    prompt_template_config = PromptTemplateConfig(**data)

In [7]:
chat_function = kernel.add_function(
    prompt_template_config=prompt_template_config,
    plugin_name="ChatBot",
    function_name="Chat",
)

## ChatBot setup

In [8]:
history = ChatHistory()

In [9]:
async def ask_question(question: str, chat_history: ChatHistory) -> str:
    """Asks a question to the chatbot and returns the answer.
    
    Args:
        question (str): The question to ask the chatbot.
        chat_history (ChatHistory): The chat history object.
        
    Returns:
        str: The answer from the chatbot.
    """

    # Create important information prompt that contains the SQL database information.
    engine_specific_rules = "Use TOP X to limit the number of rows returned instead of LIMIT X. NEVER USE LIMIT X as it produces a syntax error."
    important_information_prompt = f"""
    [SQL DATABASE INFORMATION]
    {sql_plugin.system_prompt(engine_specific_rules=engine_specific_rules)}
    [END SQL DATABASE INFORMATION]
    """

    arguments = KernelArguments()
    arguments["chat_history"] = chat_history
    arguments["important_information"] = important_information_prompt
    arguments["user_input"] = question

    logging.info("Question: %s", question)

    answer = await kernel.invoke(
        function_name="Chat",
        plugin_name="ChatBot",
        arguments=arguments,
        chat_history=chat_history,
    )

    logging.info("Answer: %s", answer)

    # Log the question and answer to the chat history.
    chat_history.add_user_message(question)
    chat_history.add_message({"role": "assistant", "message": answer})

    json_answer = json.loads(str(answer))

    display(Markdown(json_answer["answer"]))

In [10]:
await ask_question("What are the different product categories we have?", history)

INFO:root:Question: What are the different product categories we have?
INFO:semantic_kernel.functions.kernel_function:Function ChatBot-Chat invoking.
INFO:semantic_kernel.contents.chat_history:Could not parse prompt <message role="system">
As a senior analyst, your primary responsibility is to provide precise and thorough answers to the user's queries. Utilize all the provided functions to craft your responses. You must deliver detailed and accurate final answers with clear explanations and actionable insights.

Always use the provided functions to obtain key information in order to answer the question.
If you are asked to use always use a function, you must use that function to compliment the answer.
Always use multiple functions to formulate the answer.
Always execute multiple functions in parallel to compliment the results.

The response to the user must meet the requirements in RESPONSE OUTPUT REQUIREMENTS.
IMPORTANT INFORMATION contains useful information that you can use to aid y

### Product Categories and Subcategories

The product categories and their corresponding subcategories in our database are as follows:

#### Accessories
- Bike Racks
- Bike Stands
- Bottles and Cages
- Cleaners
- Fenders
- Helmets
- Hydration Packs
- Lights
- Locks
- Panniers
- Pumps
- Tires and Tubes

#### Clothing
- Bib-Shorts
- Caps
- Gloves
- Jerseys
- Shorts
- Socks
- Tights
- Vests

#### Components
- Handlebars
- Bottom Brackets
- Brakes
- Chains
- Cranksets
- Derailleurs
- Forks
- Headsets
- Mountain Frames
- Pedals
- Road Frames
- Saddles
- Touring Frames
- Wheels

#### Bikes
- Mountain Bikes
- Road Bikes
- Touring Bikes

These categories and subcategories help in organising and categorising our products effectively, facilitating a better understanding and management of our inventory.

In [11]:
await ask_question("What is the top performing product by quantity of units sold?", history)

INFO:root:Question: What is the top performing product by quantity of units sold?
INFO:semantic_kernel.functions.kernel_function:Function ChatBot-Chat invoking.
INFO:semantic_kernel.contents.chat_history:Could not parse prompt <message role="system">
As a senior analyst, your primary responsibility is to provide precise and thorough answers to the user's queries. Utilize all the provided functions to craft your responses. You must deliver detailed and accurate final answers with clear explanations and actionable insights.

Always use the provided functions to obtain key information in order to answer the question.
If you are asked to use always use a function, you must use that function to compliment the answer.
Always use multiple functions to formulate the answer.
Always execute multiple functions in parallel to compliment the results.

The response to the user must meet the requirements in RESPONSE OUTPUT REQUIREMENTS.
IMPORTANT INFORMATION contains useful information that you can u

The top performing product by quantity of units sold is the **Classic Vest, S**, with a total of 87 units sold [1][2].

In [12]:
await ask_question("Which country did we sell the most to in June 2008?", history)

INFO:root:Question: Which country did we sell the most to in June 2008?
INFO:semantic_kernel.functions.kernel_function:Function ChatBot-Chat invoking.
INFO:semantic_kernel.contents.chat_history:Could not parse prompt <message role="system">
As a senior analyst, your primary responsibility is to provide precise and thorough answers to the user's queries. Utilize all the provided functions to craft your responses. You must deliver detailed and accurate final answers with clear explanations and actionable insights.

Always use the provided functions to obtain key information in order to answer the question.
If you are asked to use always use a function, you must use that function to compliment the answer.
Always use multiple functions to formulate the answer.
Always execute multiple functions in parallel to compliment the results.

The response to the user must meet the requirements in RESPONSE OUTPUT REQUIREMENTS.
IMPORTANT INFORMATION contains useful information that you can use to aid 

The country with the highest sales in June 2008 was the United Kingdom, with a total sales amount of £572,496.56 [1].